## Variables

### Import des librairies

In [4]:
import os
import code.filemanagement as fm
import code.wikidata as wd
import code.ontorefine as otr
import code.graphdb as gd
import code.graphrdf as gr
import code.strprocessing as sp
from rdflib import Graph, Namespace, URIRef, Literal, BNode, XSD

### Définition des variables globales

In [5]:
# Fichiers existants
addr_ont_file_name = "address_ont.ttl"
ev_ont_file_name = "events_ont.ttl"

# Fichiers créés durant le processus
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"

tmp_folder_name = "tmp_files"
source_folder_name = "sources"
mapping_folder_name = "mappings"
wikidata_folder_name = "wikidata"

project_name = "voies_paris_hist" 
graphdb_url = "http://localhost:7200"
local_uri = "http://rdf.geohistoricaldata.org/address#"

wd_graph_name = "wikidata"
addr_graph_name = "local_street"

### Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier temporaire s'il n'existe pas encore pour stocker des fichiers à vocation d'être supprimés

In [6]:
tmp_folder = os.path.abspath(tmp_folder_name)
wikidata_folder = os.path.join(tmp_folder, wikidata_folder_name)
source_folder = os.path.abspath(source_folder_name)
mapping_folder = os.path.abspath(mapping_folder_name)

fm.create_folder_if_not_exists(tmp_folder)
fm.create_folder_if_not_exists(wikidata_folder)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
addr_ont_file = os.path.abspath(addr_ont_file_name)
ev_ont_file = os.path.abspath(ev_ont_file_name)

## Gestion du répertoire local

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait

In [7]:
gd.create_config_local_repository_file(local_config_file, project_name)
gd.create_repository_from_config_file(graphdb_url, local_config_file)

{"message":"Repository voies_paris_hist already exists."}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1649    0    57  100  1592    887  24775 --:--:-- --:--:-- --:--:-- 25765


### Vidage du répertoire local
Le répertoire dont l'id est `project_name` pour y stocker les données récupérées, cela est utile si le répertoire existait déjà.

In [8]:
# gd.clear_repository(graphdb_url, project_name)

### Import des ontologies

In [9]:
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, addr_ont_file)
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, ev_ont_file)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19366    0     0  100 19366      0   112k --:--:-- --:--:-- --:--:--  112k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20197    0     0  100 20197      0   8547  0:00:02  0:00:02 --:--:--  8550


''

## Import des données de Wikidata
Récupération des fichiers ttl d'éléments de Wikidata (voies, quartiers, arrondissements de Paris). Ensuite ces fichiers sont importés dans le répertoire local.

### Récupération des éléments de Wikidata
Pour chaque élément Wikidata décrivant une rue de Paris, on extrait un fichier `ttl` le décrivant.
Ce processus peut prendre du temps (environ 1 heure et demie) du fait du nombre d'éléments à récupérer (presque 7000).

#### Création des requêtes
Création des requêtes pour récupérer les données liées aux voies de Paris et une autre pour avoir les zones de la ville (quartiers administratifs et arrondissements)

In [10]:
qid_variable = "qid"

query_streets = f"""
SELECT DISTINCT ?{qid_variable}
WHERE {{
{{?{qid_variable} p:P361 [ps:P361 wd:Q16024163].}}
UNION
{{?{qid_variable} p:P361 [ps:P361 wd:Q107311481].}}
}}
ORDER BY ?{qid_variable}
"""

query_areas = f"""
SELECT DISTINCT ?qid WHERE {{
  {{BIND (wd:Q90 AS ?{qid_variable}) }}
  UNION{{?{qid_variable} p:P31 [ps:P31 wd:Q252916].}}
  UNION{{?{qid_variable} p:P31 [ps:P31 wd:Q702842]; p:P131 [ps:P131 wd:Q90].}}
}}
"""

#### Lancement des requêtes

In [11]:
# # Décommenter ces lignes pour récupérer les données

# # Récupération des voies de Paris
# wd.get_ttl_files_from_wikidata_query(query_streets, qid_variable, wikidata_folder, flavor="dump")

# #Récupération des zones de Paris
# wd.get_ttl_files_from_wikidata_query(query_areas, qid_variable, wikidata_folder, flavor="dump")

### Import des fichiers `ttl` venant de Wikidata
Importer tous les fichiers ttl dans un graphe nommé défini par `wd_graph_name` dans le répertoire (une vingtaine de minutes pour 7000 fichiers)

In [12]:
# gd.upload_ttl_folder_in_graphdb_repository(wikidata_folder, graphdb_url, project_name, wd_graph_name)

## Traitements pour créer / importer des données

### Requêtes SPARQL pour construire le graphe des rues de Paris à partir de Wikidata

:warning: Impossible de chaîner les requêtes sur le endpoint de Wikidata, d'où la séparation de celles-ci sous plusieurs variables

Note : `wd:Q16024163` décrit l'élément `réseau viaire de Paris` donc le triplet `?street p:P361 [ps:P361 wd:Q16024163].` permet de sélectionner les élements qui font partie de ce réseau. `wd:Q107311481` est similaire à la différence qu'il décrit l'élément `anciennes voies de Paris`.

* `query_1` : création des voies de Paris via un élément Wikidata décrivant une voie du réseau, initialisation d'une ressource décrivant un attribut de type nom
* `query_2` : sélection des voies de Paris avec l'ensemble de leur nom officiel avec si elles existent les dates de début et de fin de validité du nom officiel pour la voie
* `query_3` : sélection (si elles existent) des dates de création et de disparition des voies de Paris
* `query_4` : similaire à `query_1` mais repose sur les quartiers et arrondissement de Paris
* `query_5` : ajout de la localisation des entités géographiques
* `query_6` : ajout de données annexes aux entités géographiques (label, alias)


In [13]:
# Récupération de l'URI des graphes à partir de leur nom
wd_graph_name_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, wd_graph_name)
addr_graph_name_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, addr_graph_name)
local_uri = "http://rdf.geohistoricaldata.org/address#"

# Récupération des préfixes pour les requêtes
# Ajout du préfixe `ofn` au cas où il existe pas...
perso_namespaces = {"ofn":"http://www.ontotext.com/sparql/functions/"}
prefixes = gd.get_repository_prefixes(graphdb_url, project_name, perso_namespaces)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2464    0  2464    0     0   355k      0 --:--:-- --:--:-- --:--:--  401k


In [14]:
# Structurer les rues de Paris, et les zones selon notre modèle

# Vider le graphe `addr_graph_name_uri`
query0 = prefixes + f"""
DELETE {{ ?s ?p ?o }}
WHERE {{ GRAPH <{addr_graph_name_uri}> {{ ?s ?p ?o }} }}
"""

# Création des rues de Paris et ajout d'un lien owl:sameAs avec son équivalent sur Wikidata
query1 = prefixes + f"""
INSERT {{
  GRAPH <{addr_graph_name_uri}> {{
    ?landmark a addr:Landmark;
            addr:isLandmarkType addr:Thoroughfare;
            addr:hasAttribute ?attrName;
            owl:sameAs ?street.
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute.
  }}
}}
WHERE {{
  {{
    SELECT * WHERE {{
      {{ ?street p:P361 [ps:P361 wd:Q16024163]. }}
      UNION
      {{ ?street p:P361 [ps:P361 wd:Q107311481]. }}
    }}
  }}
  BIND(URI(CONCAT("{local_uri}LM_", STRUUID())) AS ?landmark)
  BIND(URI(CONCAT("{local_uri}AN_", STRUUID())) AS ?attrName)
  }}
"""

# Obtention des noms officiels des rues
query2 = prefixes + f"""
INSERT {{
  GRAPH <{addr_graph_name_uri}> {{
    ?attrName addr:version ?attrNameVersion.
    ?attrNameVersion owl:sameAs ?officialNameSt; a addr:AttributeVersion; addr:value ?officialName.
    ?startEvent a addr:Event; addr:hasTime ?startTimeInstant.
    ?endEvent a addr:Event; addr:hasTime ?endTimeInstant.
    ?startChange a addr:AttributeChange; addr:isChangeType addr:NameChange; addr:dependsOn ?startEvent; addr:appliedTo ?attrName; addr:after ?attrNameVersion.
    ?endChange a addr:AttributeChange; addr:isChangeType addr:NameChange; addr:dependsOn ?endEvent; addr:appliedTo ?attrName; addr:before ?attrNameVersion.
    ?startTimeInstant a addr:TimeInstant; addr:timeStamp ?startDateValue; addr:timePrecision ?startDatePrec; addr:timeCalendar ?startDateCal.
    ?endTimeInstant a addr:TimeInstant; addr:timeStamp ?endDateValue; addr:timePrecision ?endDatePrec; addr:timeCalendar ?endDateCal.
  }}
}}
WHERE {{
  {{
    SELECT * WHERE {{
      ?landmark a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; owl:sameAs [p:P1448 ?officialNameSt]; addr:hasAttribute ?attrName.
      ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute.
      ?officialNameSt ps:P1448 ?officialName.
      OPTIONAL{{?officialNameSt pqv:P580 [wikibase:timeValue ?startDateValue; wikibase:timePrecision ?startDatePrec; wikibase:timeCalendarModel ?startDateCal]}}
      OPTIONAL{{?officialNameSt pqv:P582 [wikibase:timeValue ?endDateValue; wikibase:timePrecision ?endDatePrec; wikibase:timeCalendarModel ?endDateCal]}}
      FILTER (LANG(?officialName) = "fr")
    }}
  }}
  BIND(URI(CONCAT("{local_uri}ANV_", STRUUID())) AS ?attrNameVersion)
  BIND(URI(CONCAT("{local_uri}EV_", STRUUID())) AS ?startEvent)
  BIND(URI(CONCAT("{local_uri}CH_", STRUUID())) AS ?startChange)
  BIND(URI(CONCAT("{local_uri}EV_", STRUUID())) AS ?endEvent)
  BIND(URI(CONCAT("{local_uri}CH_", STRUUID())) AS ?endChange)
  BIND(URI(CONCAT("{local_uri}TE_", STRUUID())) AS ?startTimeInstant)
  BIND(URI(CONCAT("{local_uri}TE_", STRUUID())) AS ?endTimeInstant)
}}
"""

# Obtention des dates de création et des dissolution des rues
query3 = prefixes + f"""
INSERT {{
  GRAPH <{addr_graph_name_uri}> {{
    ?startEvent a addr:Event; addr:hasTime ?startTimeInstant.
    ?endEvent a addr:Event; addr:hasTime ?endTimeInstant.
    ?startChange a addr:LandmarkChange; addr:isChangeType addr:LandmarkCreation; addr:appliedTo ?landmark; addr:dependsOn ?startEvent.
    ?endChange a addr:LandmarkChange; addr:isChangeType addr:LandmarkDissolution; addr:appliedTo ?landmark; addr:dependsOn ?endEvent.
    ?startTimeInstant a addr:TimeInstant; addr:timeStamp ?creationTimeValue; addr:timePrecision ?creationTimePrec; addr:timeCalendar ?creationTimeCal.
    ?endTimeInstant a addr:TimeInstant; addr:timeStamp ?dissolutionTimeValue; addr:timePrecision ?dissolutionTimePrec; addr:timeCalendar ?dissolutionTimeCal.
  }}
}}
WHERE {{
  {{
    SELECT * WHERE {{
      ?landmark a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; owl:sameAs ?street.
      OPTIONAL {{ ?street p:P571 [psv:P571 [wikibase:timeValue ?creationTimeValue; wikibase:timePrecision ?creationTimePrec; wikibase:timeCalendarModel ?creationTimeCal]]. }}
      OPTIONAL {{ ?street p:P576 [psv:P576 [wikibase:timeValue ?dissolutionTimeValue; wikibase:timePrecision ?dissolutionTimePrec; wikibase:timeCalendarModel ?dissolutionTimeCal]]. }}
    }}
  }}
  BIND(URI(CONCAT("{local_uri}EV_", STRUUID())) AS ?startEvent)
  BIND(URI(CONCAT("{local_uri}CH_", STRUUID())) AS ?startChange)
  BIND(URI(CONCAT("{local_uri}EV_", STRUUID())) AS ?endEvent)
  BIND(URI(CONCAT("{local_uri}CH_", STRUUID())) AS ?endChange)
  BIND(URI(CONCAT("{local_uri}TE_", STRUUID())) AS ?startTimeInstant)
  BIND(URI(CONCAT("{local_uri}TE_", STRUUID())) AS ?endTimeInstant)
}}
"""

# Données relatives aux arrondissements, aux quartiers de Paris et à la ville
query4 = prefixes + f"""
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
  ?landmark a addr:Landmark;
           addr:isLandmarkType ?landmarkType;
           owl:sameAs ?qid.
  }}
}}
WHERE {{
  {{
    SELECT * WHERE {{
      {{BIND (wd:Q90 AS ?qid) }}
      UNION{{?qid p:P31 [ps:P31 wd:Q252916].}}
      UNION{{?qid p:P31 [ps:P31 wd:Q702842]; p:P131 [ps:P131 wd:Q90].}}
      ?qid p:P31 [ps:P31 ?wdLandmarkType].
      BIND(
        IF(?wdLandmarkType = wd:Q252916, addr:District,
            IF(?wdLandmarkType = wd:Q252916, addr:District,
                IF(?wdLandmarkType = wd:Q484170, addr:City, ?x))) AS ?landmarkType)
      FILTER(BOUND(?landmarkType))
    }}
  }}
    BIND(URI(CONCAT("{local_uri}LM_", STRUUID())) AS ?landmark)
}}
"""

# Obtention des localisations des objets (rues, quartiers et arrondissements)
query5 = prefixes + f"""
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
  ?landmarkRelation a addr:LandmarkRelation; addr:isLandmarkRelationType addr:Within; addr:locatum ?landmark; addr:relatum ?loc.
  ?startEvent a addr:Event.
  ?endEvent a addr:Event.
  ?startChange a addr:LandmarkRelationChange; addr:isChangeType addr:RelationCreation; addr:appliedTo ?landmarkRelation; addr:dependsOn ?startEvent.
  ?endChange a addr:LandmarkRelationChange; addr:isChangeType addr:RelationDissolution; addr:appliedTo ?landmarkRelation; addr:dependsOn ?endEvent.
  }} 
}}
WHERE {{
  {{
    SELECT * WHERE {{
      ?landmark owl:sameAs ?wdLandmark.
      ?wdLandmark p:P131 [ps:P131 ?wdLoc].
      ?loc owl:sameAs ?wdLoc.
    }}
  }}
  BIND(URI(CONCAT("{local_uri}LMR_", STRUUID())) AS ?landmarkRelation)
  BIND(URI(CONCAT("{local_uri}EV_", STRUUID())) AS ?startEvent)
  BIND(URI(CONCAT("{local_uri}CH_", STRUUID())) AS ?startChange)
  BIND(URI(CONCAT("{local_uri}EV_", STRUUID())) AS ?endEvent)
  BIND(URI(CONCAT("{local_uri}CH_", STRUUID())) AS ?endChange)
}}
"""

# Ajout des noms des objets (labels et alias)
query6 = prefixes + f"""
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
 ?landmark rdfs:label ?wdLandmarkLabel;
           skos:altLabel ?wdLandmarkAltLabel.
}} 
}}
WHERE {{
  ?landmark a addr:Landmark; owl:sameAs ?wdLandmark.
  ?wdLandmark rdfs:label ?wdLandmarkLabel.
  FILTER (LANG(?wdLandmarkLabel) = "fr")
  OPTIONAL {{?wdLandmark skos:altLabel ?wdLandmarkAltLabel. FILTER (LANG(?wdLandmarkAltLabel) = "fr")}}
}}
"""

wiki_queries = [query0, query1, query2, query3, query4, query5, query6]

Création d'un graphe nommé stockant les rues selon l'ontologie définie à partir des données de Wikidata

In [16]:
for query in wiki_queries:
    gd.update_query(query, graphdb_url, project_name)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3993    0     0  100  3993      0    124  0:00:32  0:00:32 --:--:--     0
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4897    0     0  100  4897      0   5702 --:--:-- --:--:-- --:--:--  5700
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7058    0     0  100  7058      0   1124  0:00:06  0:00:06 --:--:--     0
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6433    0     0  100  6433      0   1365  0:00:04  0:00:04 --:--:--     0
  % Total    % Received % Xferd  Average Speed   Tim

#### Traitements après l'import de données

* `loc_query_1` : remplace les précisions (liées à `addr:timePrecision`) qui sont des entiers (voir https://www.wikidata.org/wiki/Help:Dates/fr) par une ressource de l'ontologie OWL Time (des sous-classes de `time:TemporalUnit`)
* `loc_query_2` : lors de la construction du graphe via les requêtes de la section précédente, chaque version des noms (`nomVersion`) est associée à deux changements définissant son début et sa fin de validité (`changementDebut` et `changementFin`) qui dépendent chacun d'un événement (`evenementDebut` et `evenementFin`). Lorsqu'une voie change de nom, soit passe de `nomVersion1` à `nomVersion2`, `changementFin1` et `changementDebut2` définissent la même chose (la fin de validité de `nomVersion1` implique le début de validité de `nomVersion2`). Cette requête permet de faire la fusion de ces changements et ainsi des événéments qui en dépendent.
* `loc_query_3` : similaire à la requête `loc_query_2` mais s'applique aux repères ;
* `loc_query_4` : suppression d'éléments de type `addr:TimeInstant` qui n'ont pas de propriété `addr:timeStamp`.

In [17]:
# Remplacement de la précision donnée par un entier (voir https://www.wikidata.org/wiki/Help:Dates/fr) par une ressource de l'ontologie OWL Time
loc_query_1 = prefixes + f"""
DELETE {{ 
    ?timeInstant addr:timePrecision ?timePrecision.
}}
INSERT {{
    GRAPH <{addr_graph_name_uri}> {{
        ?timeInstant addr:timePrecision ?timeUnit.
    }} 
}}
WHERE {{ 
    GRAPH <{addr_graph_name_uri}> {{
        ?timeInstant a addr:TimeInstant; addr:timePrecision ?timePrecision.
        FILTER (DATATYPE(?timePrecision) = xsd:integer)
            BIND(IF(?timePrecision = 11, time:unitDay, 
                    IF(?timePrecision = 10, time:unitMonth,
                        IF(?timePrecision = 9, time:unitYear,
                            IF(?timePrecision = 8, time:unitDecade,
                                IF(?timePrecision = 7, time:unitCentury,
                                    IF(?timePrecision = 6, time:unitMillenium, ?x
                                )))))) AS ?timeUnit)
    }} 
}}
"""

loc_query_2 = prefixes + f"""
DELETE {{
    ?change1 ?p ?o.
    ?event1 ?pe ?oe.
    ?se ?pe ?event1.
    ?timeInstant1 a addr:timeInstant; addr:timeStamp ?timeStamp1; addr:timePrecision ?timePrec1; addr:timeCalendar ?timeCal1.
}}
INSERT {{
    GRAPH <{addr_graph_name_uri}> {{
        ?change2 addr:before ?attrNameVersion1.
    }}
}}
WHERE {{
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion1, ?attrNameVersion2.
    ?attrNameVersion1 a addr:AttributeVersion.
    ?attrNameVersion2 a addr:AttributeVersion.
    FILTER (?attrNameVersion1 != ?attrNameVersion2)
    ?change1 a addr:AttributeChange; addr:before ?attrNameVersion1; addr:dependsOn ?event1.
    ?event1 a addr:Event; addr:hasTime ?timeInstant1. 
    ?timeInstant1 a addr:TimeInstant; addr:timeStamp ?timeStamp1; addr:timePrecision ?timePrec1; addr:timeCalendar ?timeCal1.
    ?change2 a addr:AttributeChange; addr:after ?attrNameVersion2; addr:dependsOn [a addr:Event; addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeStamp2; addr:timePrecision ?timePrec2; addr:timeCalendar ?timeCal2]].
    BIND(ofn:asDays(?timeStamp2 - ?timeStamp1) AS ?diffTime)
    FILTER(?diffTime >= 0.0 && ?timeCal1 = ?timeCal2)
    FILTER ((?diffTime <= 1.0 && ?timePrec1 = time:unitDay && ?timePrec2 = time:unitDay) ||
         (?diffTime <= 0 && ?timePrec1 != time:unitDay && ?timePrec2 != time:unitDay))
    ?change1 ?p ?o.
    {{?event1 ?pe ?oe}}UNION{{?se ?pe ?event1}}
}}
"""

loc_query_3 = prefixes + f"""
DELETE {{
    ?creaEv ?p ?o.
    ?s ?p ?creaEv.
    ?crevEvTime a addr:TimeInstant; addr:timePrecision ?creaEvTimePrec; addr:timeStamp ?creaEvTimeVal; addr:timeCalendar ?creaEvTimeCal.
}}
INSERT {{ GRAPH <{addr_graph_name_uri}> {{
    ?lmCrea addr:dependsOn ?attrNameEv.
}} }}
WHERE {{
    ?lmCrea a addr:LandmarkChange; addr:isChangeType addr:LandmarkCreation; addr:appliedTo ?landmark; addr:dependsOn ?creaEv.
    ?creaEv a addr:Event; addr:hasTime ?crevEvTime.
    ?crevEvTime a addr:TimeInstant; addr:timePrecision ?creaEvTimePrec; addr:timeStamp ?creaEvTimeVal; addr:timeCalendar ?creaEvTimeCal.
    ?landmark a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?nameAttr.
    ?attrNameChange a addr:AttributeChange; addr:appliedTo ?nameAttr; addr:dependsOn ?attrNameEv.
    ?attrNameEv a addr:Event; addr:hasTime ?attrNameEvTime. 
    ?attrNameEvTime a addr:TimeInstant; addr:timePrecision ?newNameEvTimePrec; addr:timeStamp ?newNameEvTimeVal; addr:timeCalendar ?newNameEvTimeCal.
    FILTER(?attrNameEv != ?creaEv)
    BIND(ofn:asDays(?creaEvTimeVal - ?newNameEvTimeVal) AS ?diffTime)
    FILTER(?diffTime >= 0.0 && ?newNameEvTimeCal = ?creaEvTimeCal)
    FILTER ((?diffTime <= 1.0 && ?creaEvTimePrec = time:unitDay && ?newNameEvTimePrec = time:unitDay) ||
         (?diffTime <= 0 && ?creaEvTimePrec != time:unitDay && ?timePrec2 != time:newNameEvTimePrec))
        {{?creaEv ?p ?o}}UNION{{?s ?p ?creaEv}}
}}
"""

loc_query_4 = prefixes + f"""
DELETE {{
    ?timeInstant ?p ?o.
    ?s ?p ?timeInstant.
}}
WHERE {{
    ?timeInstant a addr:TimeInstant.
    OPTIONAL {{ ?timeInstant addr:timeStamp ?timeStamp }}
    FILTER (!BOUND(?timeStamp))
    {{?timeInstant ?p ?o}}UNION{{?s ?p ?timeInstant}}
}}
"""

loc_queries = [loc_query_1, loc_query_2, loc_query_3, loc_query_4]

Nettoyage des données en fusionnant certains événements et changements

In [18]:
for loc_query in loc_queries:
    gd.update_query(loc_query, graphdb_url, project_name)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5469    0     0  100  5469      0   2365  0:00:02  0:00:02 --:--:--  2365
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6084    0     0  100  6084      0   1883  0:00:03  0:00:03 --:--:--  1883
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6013    0     0  100  6013      0  11197 --:--:-- --:--:-- --:--:-- 11197
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4216    0     0  100  4216      0    563  0:00:07  0:00:07 --:--:--     0


### Requêtes pour faire des sélections dans le graphe

In [19]:
# Sélection des noms qu'a pris la place de la Nation durant sa vie
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?street ?name ?timeValueStart ?timeValueEnd WHERE { 
	?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attrName; (rdfs:label|skos:altLabel) "place de la Nation"@fr.
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion.
    ?attrNameVersion addr:value ?name.
    ?changeEnd addr:before ?attrNameVersion; addr:appliedTo ?attrName; addr:dependsOn ?eventEnd.
    ?changeStart addr:after ?attrNameVersion; addr:appliedTo ?attrName; addr:dependsOn ?eventStart.
    ?eventEnd a addr:Event.
    ?eventStart a addr:Event.
    OPTIONAL {?eventEnd addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValueEnd]}
    OPTIONAL {?eventEnd addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecisionEnd]}
    OPTIONAL {?eventEnd addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendarEnd]}
    OPTIONAL {?eventStart addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValueStart]}
    OPTIONAL {?eventStart addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecisionStart]}
    OPTIONAL {?eventStart addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendarStart]}
    }
""" 

# Sélection de l'historique des noms de la place de la Nation avec les dates
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?street ?change ?event ?timeValue ?timePrecision ?timeCalendar ?beforeVersion ?afterVersion WHERE { 
	?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attrName; (rdfs:label|skos:altLabel) "place de la Nation"@fr.
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute.
    ?change addr:appliedTo ?attrName; addr:dependsOn ?event.
    ?event a addr:Event.
    OPTIONAL {?event addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue]}
    OPTIONAL {?event addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecision]}
    OPTIONAL {?event addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendar]}
    OPTIONAL {?change addr:before [addr:value ?beforeVersion]}
    OPTIONAL {?change addr:after [addr:value ?afterVersion]}
}
"""

# Sélection du nom de la place de la Nation à une date donnée
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?street ?name ?selectTime WHERE { 
	?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attrName; (rdfs:label|skos:altLabel) "place de la Nation"@fr.
    BIND("1800-01-08T00:00:00"^^xsd:dateTime AS ?selectTime)
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion.
    ?attrNameVersion a addr:AttributeVersion; addr:value ?name.
    ?change1 addr:before ?attrNameVersion; addr:dependsOn ?event1.
    ?change2 addr:after ?attrNameVersion; addr:dependsOn ?event2.
    ?event1 a addr:Event.
    ?event2 a addr:Event.
    OPTIONAL {?event1 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue1]}
    OPTIONAL {?event1 addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecision1]}
    OPTIONAL {?event1 addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendar1]}
    OPTIONAL {?event2 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue2]}
    OPTIONAL {?event2 addr:hasTime [a addr:TimeInstant; addr:timePrecision ?timePrecision2]}
    OPTIONAL {?event2 addr:hasTime [a addr:TimeInstant; addr:timeCalendar ?timeCalendar2]}
    FILTER ((?timeValue1 >= ?selectTime && ?timeValue2 <= ?selectTime && ?timeCalendar1 = ?timeCalendar2) ||
        (?timeValue1 >= ?selectTime && !BOUND(?timeValue2)) ||
        (?timeValue2 <= ?selectTime && !BOUND(?timeValue1))
    ) 
}
""" 

# Sélection des voies existant à une date donnée avec leur nom d'époque
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?street ?name ?timeValueStreetCrea ?timeValueStreetDiss WHERE { 
    BIND("1900-01-08T00:00:00"^^xsd:dateTime AS ?selectTime)
	?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attrName.
    ?changeStreetDiss addr:dependsOn ?eventStreetDiss; addr:isChangeType addr:LandmarkDissolution; addr:appliedTo ?street.
    ?changeStreetCrea addr:dependsOn ?eventStreetCrea; addr:isChangeType addr:LandmarkCreation; addr:appliedTo ?street.
    ?eventStreetDiss a addr:Event.
    ?eventStreetCrea a addr:Event.
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion.
    ?attrNameVersion a addr:AttributeVersion; addr:value ?name.
    ?change1 addr:before ?attrNameVersion; addr:dependsOn ?event1.
    ?change2 addr:after ?attrNameVersion; addr:dependsOn ?event2.
    ?event1 a addr:Event.
    ?event2 a addr:Event.
    OPTIONAL {?event1 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue1; addr:timePrecision ?timePrecision1; addr:timeCalendar ?timeCalendar1]}
    OPTIONAL {?event2 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue2; addr:timePrecision ?timePrecision2; addr:timeCalendar ?timeCalendar2]}
    OPTIONAL {?eventStreetDiss addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValueStreetDiss; addr:timePrecision ?timePrecisionStreetDiss; addr:timeCalendar ?timeCalendarStreetDiss]}
    OPTIONAL {?eventStreetCrea addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValueStreetCrea; addr:timePrecision ?timePrecisionStreetCrea; addr:timeCalendar ?timeCalendarStreetCrea]}
    FILTER ((?timeValue1 >= ?selectTime && ?timeValue2 <= ?selectTime && ?timeCalendar1 = ?timeCalendar2) ||
        (?timeValue1 >= ?selectTime && !BOUND(?timeValue2)) ||
        (?timeValue2 <= ?selectTime && !BOUND(?timeValue1)) ||
        (!BOUND(?timeValue1) && !BOUND(?timeValue2))
    ) 
    FILTER ((?timeValueStreetDiss >= ?selectTime && ?timeValueStreetCrea <= ?selectTime && ?timeCalendarStreetDiss = ?timeCalendarStreetCrea) ||
        (?timeValueStreetDiss >= ?selectTime && !BOUND(?timeValueStreetCrea)) ||
        (?timeValueStreetCrea <= ?selectTime && !BOUND(?timeValueStreetDiss)) ||
        (!BOUND(?timeValueStreetDiss) && !BOUND(?timeValueStreetCrea))
    ) 
}
"""

Une fois une base de graphe créée, insertion de nouvelles données décrivant des états :
* création d'un graphe nommé décrivant l'état du territoire à l'instant donné par `timestamp`
* création de liens entre les ressources du graphe nommé et celui qui décrit l'état du territoire au même instant via une source externe.

In [20]:
def get_queries_for_sources(graph_year_uri, graph_state_uri, timestamp, prefixes):
    # Pour chaque voie existant à l'instant défini par le timestamp, création d'une version
    query1 = prefixes + f"""
    INSERT {{
        GRAPH <{graph_year_uri}> {{
        ?version a addr:LandmarkVersion; addr:versionOf ?street.
            }}
    }}
    WHERE {{
        GRAPH <{addr_graph_name_uri}> {{
            {{
                SELECT * WHERE {{
                    BIND("{timestamp}"^^xsd:dateTime AS ?selectTime)
                    ?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare.
                    ?change1 addr:dependsOn ?event1; addr:isChangeType addr:LandmarkDissolution; addr:appliedTo ?street.
                    ?change2 addr:dependsOn ?event2; addr:isChangeType addr:LandmarkCreation; addr:appliedTo ?street.
                    ?event1 a addr:Event.
                    ?event2 a addr:Event.
                    OPTIONAL {{?event1 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue1;addr:timePrecision ?timePrecision1; addr:timeCalendar ?timeCalendar1]}}
                    OPTIONAL {{?event2 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue2; addr:timePrecision ?timePrecision2; addr:timeCalendar ?timeCalendar2]}}
                    FILTER ((?timeValue1 >= ?selectTime && ?timeValue2 <= ?selectTime && ?timeCalendar1 = ?timeCalendar2) ||
                        (?timeValue1 >= ?selectTime && !BOUND(?timeValue2)) ||
                        (?timeValue2 <= ?selectTime && !BOUND(?timeValue1)) ||
                        (!BOUND(?timeValue1) && !BOUND(?timeValue2)) 
                    )
                }}
            }}
        BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#LMV_", STRUUID())) AS ?version)
        }}
    }}
    """ 

    # Ajout des attributs aux voies valables à l'instant défini
    query2 = prefixes + f"""
    INSERT {{
        GRAPH <{graph_year_uri}> {{
        ?version addr:hasAttribute [addr:isAttributeType ?attrType; addr:value ?attrVersionValue]
    }}
    }}
    WHERE {{
        BIND("{timestamp}"^^xsd:dateTime AS ?selectTime)
    GRAPH <{graph_year_uri}> {{
        ?version a addr:LandmarkVersion; addr:versionOf ?street.
    }}
    GRAPH <{addr_graph_name_uri}> {{
        ?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attr.
        ?attr addr:isAttributeType ?attrType; addr:version ?attrVersion.
            ?attrVersion addr:value ?attrVersionValue.
        ?change1 addr:dependsOn ?event1; addr:before ?attrVersion; addr:appliedTo ?attr.
        ?change2 addr:dependsOn ?event2; addr:after ?attrVersion; addr:appliedTo ?attr.
        ?event1 a addr:Event.
        ?event2 a addr:Event.
        OPTIONAL {{?event1 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue1;addr:timePrecision ?timePrecision1; addr:timeCalendar ?timeCalendar1]}}
        OPTIONAL {{?event2 addr:hasTime [a addr:TimeInstant; addr:timeStamp ?timeValue2; addr:timePrecision ?timePrecision2; addr:timeCalendar ?timeCalendar2]}}
        FILTER ((?timeValue1 >= ?selectTime && ?timeValue2 <= ?selectTime && ?timeCalendar1 = ?timeCalendar2) ||
            (?timeValue1 >= ?selectTime && !BOUND(?timeValue2)) ||
            (?timeValue2 <= ?selectTime && !BOUND(?timeValue1)) ||
            (!BOUND(?timeValue1) && !BOUND(?timeValue2)) 
            )
    }}
    }}
    """

    replace_function_graph_year = sp.get_lower_simplified_french_street_name_function("?streetName")
    replace_function_graph_state = sp.get_lower_simplified_french_street_name_function("?streetSourceExtName")
    query3 = prefixes + f"""
    INSERT {{
        GRAPH <{graph_year_uri}> {{
            ?street owl:sameAs ?streetSourceExt.
        }}
    }}
    WHERE {{
        GRAPH <{graph_year_uri}> {{
            ?street addr:hasAttribute [addr:isAttributeType addr:NameAttribute; addr:value ?streetName].
            BIND({replace_function_graph_year} AS ?streetNameJoin)
        }}
        GRAPH <{graph_state_uri}> {{
            ?streetSourceExt rdfs:label ?streetSourceExtName.
            BIND({replace_function_graph_state} AS ?streetSourceExtNameJoin)
        }}
        FILTER(?streetNameJoin = ?streetSourceExtNameJoin)
    }}
    """

    return [query1, query2, query3]

## Intégration d'éléments de sources extérieures à Wikidata

Cette section permet de tester l'ajout des sources externes à Wikidata qui sont hétérogènes qui représentent plusieurs types :
* état d'un territoire à un instant donné
* événement décrivant des changements sur des entités géographiques

### Création d'un graphe nommé donnant l'état du réseau à un instant donné

Pour chaque source décrivant l'état du réseau viaire de Paris sur une période donnée (pour l'instant les données sont restreintes au quartier des Arts et Métiers), on a :
* un fichier `csv` qui contient les segments des voies, le nom des segments (nom de la voie), un identifant ;
* un fichier `json` de mapping qui va convertir le fichier précédent en un graphe de connaissances selon les modalités fixées ;
* le nom du graphe nommé dans lequel sera stocké le graphe créé par le fichier de mapping via le fichier `csv` ;
* une date qui décrit un instant de validité de la source.

In [21]:
# ontorefine_cmd = "ontorefine-cli"
ontorefine_cmd = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli"
# ontorefine_cmd = "/Applications/Ontotext\ Refine.app/Contents/app/bin/ontorefine-cli"
ontorefine_url = "http://localhost:7333"

sources = [
    {"filename": "andriveau_1849_am.csv", "mapping_file":"mapping_andriveau_1849_am.json","graphname":"andriveau1849", "date":"1849-01-01"},
    {"filename": "atlas_municipal_1888_am.csv", "mapping_file":"mapping_atlas_municipal_1888_am.json","graphname":"atlasMunicipal1888", "date":"1888-01-01"},
    {"filename": "piquet_1826_am.csv", "mapping_file":"mapping_piquet_1826_am.json","graphname":"piquet1826", "date":"1826-01-01"},
    {"filename": "verniquet_1791_am.csv", "mapping_file":"mapping_verniquet_1791_am.json","graphname":"verniquet1791", "date":"1791-01-01"},
]

Pour chaque source, on fait les traitements suivants :
* conversion du fichier `csv` en un graphe de connaissances via le fichier de mapping grâce à l'invite de commandes d'Ontotext Refine, le fichier de sortie est un fichier `ttl` décrit par la variable `export_file` ;
* afin de pouvoir faire des comparaisons de noms de rues, un traitement est opéré sur les noms pour avoir une normalisation. Par exemple, *pl. de la République* devient "place de la République* ;
* import du fichier `ttl` dans GraphDB dans le graphe nommé défini par `graph_name`
* création d'un graphe nommé `graph_state` dans lequel sont représentées des versions du réseau viaire à la date fixée, il donne l'état du réseau viaire à cette date. Pour chaque voie existant à la date t (selon le graphe), on lui associe une version : `<?versionVoie addr:versionOf ?voie>` (#TODO: expliciter le lien temporel entre `?versionVoie` et `?voie`)
* en fonction d'un ou plusieurs critères (notamment le nom), création de liens entre les éléments du graphe `graph_name` et ceux de `graph_state`.

Ainsi, pour chaque élément de la source, on l'aura associé (ou pas) à une version d'une voie.

In [24]:
for source in sources:
    source_file = os.path.join(source_folder, source.get("filename"))
    mapping_file = os.path.join(mapping_folder, source.get("mapping_file"))
    export_file = os.path.join(tmp_folder, source.get("filename").replace(".csv", ".ttl"))
    graph_name = source.get("graphname")
    date = source.get("date")
    timestamp = f"{date}T00:00:00"

    graph_year_name = f"local_street_{date}"
    graph_year_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, graph_year_name)
    graph_state_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, graph_name)

    gd.remove_graph(graphdb_url, project_name, graph_name)
    gd.remove_graph(graphdb_url, project_name, graph_year_name)
    otr.get_export_file_from_ontorefine(source_file, mapping_file, export_file, ontorefine_cmd, ontorefine_url, project_name)
    sp.normalize_street_rdfs_labels_in_graph_file(export_file)
    gd.import_ttl_file_in_graphdb(graphdb_url, project_name, export_file, graph_name)

    queries = get_queries_for_sources(graph_year_uri, graph_state_uri, timestamp, prefixes)
    for query in queries:
        gd.update_query(query, graphdb_url, project_name)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0


Successfully deleted project with identifier: 2411582997316


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  102k    0     0  100  102k      0   991k --:--:-- --:--:-- --:--:--  988k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6866    0     0  100  6866      0   7603 --:--:-- --:--:-- --:--:--  7595
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6483    0     0  100  6483      0  10998 --:--:-- --:--:-- --:--:-- 11006
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6691    0     0  100  6691      0    478  0:00:13  0:00:13 --:--:--     0
  % Total    % Received % Xferd  Average Speed   Tim

Successfully deleted project with identifier: 2182045330416


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  111k    0     0  100  111k      0  1328k --:--:-- --:--:-- --:--:-- 1330k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6866    0     0  100  6866      0   8370 --:--:-- --:--:-- --:--:--  8362
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6483    0     0  100  6483      0  10981 --:--:-- --:--:-- --:--:-- 10969
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6696    0     0  100  6696      0    276  0:00:24  0:00:24 --:--:--     0
  % Total    % Received % Xferd  Average Speed   Tim

Successfully deleted project with identifier: 2671515618248


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 82314    0     0  100 82314      0  1035k --:--:-- --:--:-- --:--:-- 1030k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6866    0     0  100  6866      0   9103 --:--:-- --:--:-- --:--:--  9106
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6483    0     0  100  6483      0  12663 --:--:-- --:--:-- --:--:-- 12686
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6688    0     0  100  6688      0   1785  0:00:03  0:00:03 --:--:--  1784
  % Total    % Received % Xferd  Average Speed   Tim

Successfully deleted project with identifier: 2378248066715


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73032    0     0  100 73032      0   969k --:--:-- --:--:-- --:--:--  976k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6866    0     0  100  6866      0   9517 --:--:-- --:--:-- --:--:--  9522
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6483    0     0  100  6483      0  13464 --:--:-- --:--:-- --:--:-- 13450
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6691    0     0  100  6691      0    650  0:00:10  0:00:10 --:--:--     0


### Insertion d'événements dans le graphe

Ajout de l'événement suivant cité comme ceci dans le Wikipedia français (https://fr.wikipedia.org/wiki/Rue_Pastourelle) :
>*[la rue Pastourelle] est la réunion, depuis 1877, de deux rues situées de part et d'autre de la rue des Archives :<br>
>la « rue Groignet » ouverte en 1296 [...], et de<br>
>la « rue d'Anjou » ou « rue d'Anjou au Marais » ouverte en 1626 [...].*

Les étapes pour créer cet événément :
* **sélection des entités** géographiques citées (en fonction du nom qu'elles possédaient à l'instant de l'événément) ;
* **création des entités** qui n'existent pas ;
* détection de l'événement décrit ci-dessus pour voir s'il existe déjà ;
* **création de l'événement en l'associant à une valeur temporelle**, il est le résultat de la fusion des événements associés aux changements élémentaires (si A et B fusionnent en C, l'événement va fusionner les événements liés aux dissolutions de A et B et de la création de C qui n'ont normalement pas d'entité temporelle)

#### Définition de variables globales du traitement

In [25]:
timestamp = "1877-01-01T00:00:00"
time_precision = "year"
time_calendar = "wd:Q1985727"
street_name = "rue Pastourelle"
lang = "fr"

#### Détection des entités dans le graphe

In [26]:
def get_street_in_time_from_name(street_name, timestamp:Literal, time_precision:URIRef, time_calendar:URIRef, prefixes, lang="fr"):
    val_tRef = "?timeValueRef"
    cal_tRef = "?timeCalRef"
    cal_t1 = "?timeCal1"
    val_t1 = "?timeValue1"
    cal_t2 = "?timeCal2"
    val_t2 = "?timeValue2"
    attr_cal_t1 = "?timeCalAttr1"
    attr_val_t1 = "?timeValueAttr1"
    attr_cal_t2 = "?timeCalAttr2"
    attr_val_t2 = "?timeValueAttr2"
    street_var = "street"
    street_name_var = "street_name"
    label_var = "label"
    simplified_label_var = "simplified_label"
    simplified_street_name_var = "simplified_street_name"

    time_filter_st = sp.define_time_filter_for_sparql_query(val_tRef, cal_tRef, val_t1, cal_t1, val_t2, cal_t2, time_precision)
    time_filter_attr = sp.define_time_filter_for_sparql_query(val_tRef, cal_tRef, attr_val_t1, attr_cal_t1, attr_val_t2, attr_cal_t2, time_precision)
    simplified_street_name = sp.get_lower_simplified_french_street_name_function(f"?{street_name_var}")
    simplified_label = sp.get_lower_simplified_french_street_name_function(f"?{label_var}")
    street_name_lit = Literal(street_name, lang=lang)
    query = prefixes + f"""
    SELECT ?{street_var} ?{label_var}
    WHERE {{
        BIND({timestamp.n3()} AS {val_tRef})
        BIND({time_calendar.n3()} AS {cal_tRef})
        BIND({street_name_lit.n3()} AS ?{label_var})
        BIND({simplified_label} AS ?{simplified_label_var})
        BIND(addr:Thoroughfare AS ?landmarkType)
        GRAPH <{addr_graph_name_uri}> {{
            ?{street_var} a addr:Landmark ; addr:isLandmarkType ?landmarkType.
            ?changeSt1 addr:dependsOn ?eventSt1; addr:isChangeType addr:LandmarkDissolution; addr:appliedTo ?{street_var}.
            ?changeSt2 addr:dependsOn ?eventSt2; addr:isChangeType addr:LandmarkCreation; addr:appliedTo ?{street_var}.
            ?eventSt1 a addr:Event.
            ?eventSt2 a addr:Event.
            OPTIONAL {{?eventSt1 addr:hasTime [a addr:TimeInstant; addr:timeStamp {val_t1}; addr:timePrecision ?timePrecision1; addr:timeCalendar {cal_t1}]}}
            OPTIONAL {{?eventSt2 addr:hasTime [a addr:TimeInstant; addr:timeStamp {val_t2}; addr:timePrecision ?timePrecision2; addr:timeCalendar {cal_t2}]}}
            {time_filter_st}
            ?{street_var} addr:hasAttribute ?attrName.
            ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion.
            ?attrNameVersion a addr:AttributeVersion; addr:value ?{street_name_var}.
            BIND({simplified_street_name} AS ?{simplified_street_name_var})
            ?changeAttr1 addr:before ?attrNameVersion; addr:dependsOn ?eventAttr1.
            ?changeAttr2 addr:after ?attrNameVersion; addr:dependsOn ?eventAttr2.
            ?eventAttr1 a addr:Event.
            ?eventAttr2 a addr:Event.
            OPTIONAL {{?eventAttr1 addr:hasTime [a addr:TimeInstant; addr:timeStamp {attr_val_t1}; addr:timePrecision ?timePrecisionAttr1; addr:timeCalendar {attr_cal_t1}]}}
            OPTIONAL {{?eventAttr2 addr:hasTime [a addr:TimeInstant; addr:timeStamp {attr_val_t2}; addr:timePrecision ?timePrecisionAttr2; addr:timeCalendar {attr_cal_t2}]}}
            {time_filter_attr}
            FILTER(?{simplified_street_name_var} = ?{simplified_label_var})
    }}
    }}
    """

    results = gd.select_query_to_json(query, graphdb_url, project_name)
    streets = []

    for binding in results.get("results").get("bindings"):
        result_elem = binding.get(street_var)
        street = gr.convert_result_elem_to_rdflib_elem(result_elem)
        streets.append(street)

    return streets

def create_basic_street_with_name(landmark_uri:URIRef, street_name:str, g, lang:str="fr"):
    attribute_uri = gr.generate_uri(namespace, "ATTR")
    gr.create_landmark_with_changes(landmark_uri, street_name, lang, "Thoroughfare", g, namespace)
    gr.create_landmark_attribute(attribute_uri, landmark_uri, "NameAttribute", g, namespace)
    gr.create_attribute_version(attribute_uri, street_name, g, namespace, lang)

def get_sparql_triples_to_get_landmark_change(landmark_uri:URIRef, change_type_uri:URIRef):
    random_variable = f"?{gr.generate_uuid()}"
    return f"{random_variable} a addr:Change; addr:dependsOn ?event; addr:isChangeType {change_type_uri.n3()}; addr:appliedTo {landmark_uri.n3()}"

def get_sparql_query_to_merge_events(event_landmarks:list, time_stamp:Literal, time_calendar:URIRef, time_precision:URIRef):
    """
    `event_landmarks` is a list of dictionaries. Each dictionary must contain a URI of a landmark and the type of change related to the event :
    `{"uri":URIRef('http://www.example.org/landmark'), "change_type":URIRef('http://www.example.org/Creation')}` 
    """

    changes_list = []
    for event_lm in event_landmarks:
        line = get_sparql_triples_to_get_landmark_change(event_lm.get("uri"), event_lm.get("change_type"))
        changes_list.append(f"{{ {line} }}")
    
    changes = "UNION".join(changes_list)

    query = prefixes + f"""
    DELETE {{
        ?s ?p ?event.
        ?event ?p ?o.
    }}
    INSERT {{
        ?s ?p ?mergedEvent.
        ?mergedEvent ?p ?o.
        ?mergedEvent addr:hasTime ?temporalEntity.
        ?temporalEntity a addr:TimeInstant; addr:timeCalendar {time_calendar.n3()}; addr:timePrecision {time_precision.n3()}; addr:timeStamp {time_stamp.n3()}
    }}
    WHERE {{
        {{SELECT * WHERE {{
        BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#EV_", STRUUID())) AS ?mergedEvent)
        BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#TE_", STRUUID())) AS ?temporalEntity)
            }} }}
        {changes}
        {{?s ?p ?event }} UNION {{?event ?p ?o }}
    }}
    """

    return query

g = Graph()
namespace = Namespace(local_uri)
owl_time_ns = Namespace("http://www.w3.org/2006/time#")
wd_ns = Namespace("http://www.wikidata.org/entity/")

test_graph_name = "test"
test_file_name = "test.ttl"
test_file = os.path.join(tmp_folder, test_file_name)

event_description = {"time":
                     {"stamp":Literal("1877-01-01T00:00:00",datatype=XSD.dateTime),
                      "precision":owl_time_ns["unitYear"],
                      "calendar":wd_ns["Q1985727"]},
                     "lang":"fr",
                     "landmarks":[
                         {"name":"rue d'Anjou", "change_type":"LandmarkDissolution", "landmark_type":"Thoroughfare"},
                         {"name":"rue Groignet", "change_type":"LandmarkDissolution", "landmark_type":"Thoroughfare"},
                         {"name":"rue Pastourelle", "change_type":"LandmarkCreation", "landmark_type":"Thoroughfare"}
                         ]
                         }


event_description = {"time":
                     {"stamp":Literal("1851-02-18T00:00:00",datatype=XSD.dateTime),
                      "precision":owl_time_ns["unitDay"],
                      "calendar":wd_ns["Q1985727"]},
                     "lang":"fr",
                     "landmarks":[
                         {"name":"rue Frépillon", "change_type":"LandmarkDissolution", "landmark_type":"Thoroughfare"},
                         {"name":"rue de la Croix", "change_type":"LandmarkDissolution", "landmark_type":"Thoroughfare"},
                         {"name":"rue du Pont-aux-Biches-Saint-Martin", "change_type":"LandmarkDissolution", "landmark_type":"Thoroughfare"},
                         {"name":"rue Volta", "change_type":"LandmarkCreation", "landmark_type":"Thoroughfare"}
                         ]
                         }


timestamp = event_description.get("time").get("stamp")
time_precision = event_description.get("time").get("precision")
time_calendar = event_description.get("time").get("calendar")
for landmark in event_description.get("landmarks"):
    street_name = landmark.get("name")
    streets = get_street_in_time_from_name(street_name, timestamp, time_precision, time_calendar, prefixes)

    if streets == []:
        landmark_uri = gr.generate_uri(namespace, "LM")
        landmark["uri"] = landmark_uri
        create_basic_street_with_name(landmark_uri, street_name, g)
    else:
        landmark["uri"] = streets[0]

    landmark["change_type"] = namespace[landmark.get("change_type")]

g.serialize(test_file)
gd.remove_graph(graphdb_url, project_name, test_graph_name)
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, test_file, test_graph_name)
time_stamp = Literal(timestamp, datatype=XSD.dateTime)
time_calendar = URIRef(time_calendar)
time_precision = URIRef(time_precision)                   
query = get_sparql_query_to_merge_events(event_description.get("landmarks"), time_stamp, time_calendar, time_precision)
gd.update_query(query, graphdb_url, project_name)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11125    0   113  100 11012    773  75370 --:--:-- --:--:-- --:--:-- 75680
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11126    0   113  100 11013    682  66495 --:--:-- --:--:-- --:--:-- 67430
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11144    0   113  100 11031    821  80196 --:--:-- --:--:-- --:--:-- 81343
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11418    0   415  100 11003   2697  71516 --:--:-- --:--:-- --:--:-- 74627
  % Total    % Received % Xferd  Average Speed   Tim

Ajout d'une validité temporelle pour l'appartenance aux arrondissements et aux quartiers administratifs (début : 16 juin 1859)

In [27]:
time_stamp = Literal("1859-06-16T00:00:00",datatype=XSD.dateTime)

query = prefixes + f"""
DELETE {{
    ?s ?p ?changeEvent.
    ?changeEvent ?p ?o.
}}
INSERT {{
    ?s ?p ?event.
    ?event ?p ?o.
    ?event a addr:Event; addr:hasTime ?timeInstant.
    ?timeInstant a addr:TimeInstant; addr:timeStamp {time_stamp.n3()}; addr:timePrecision time:unitDay; addr:timeCalendar {time_calendar.n3()}.
}}
WHERE {{
    ?lmr a addr:LandmarkRelation;
         addr:isLandmarkRelationType addr:Within;
         addr:relatum [a addr:Landmark; addr:isLandmarkType addr:District].
    ?change a addr:Change ; addr:isChangeType addr:RelationCreation ; addr:appliedTo ?lmr; addr:dependsOn ?changeEvent.
    {{
        SELECT * WHERE {{
            BIND(URI(CONCAT("{local_uri}EV_", STRUUID())) AS ?event)
            BIND(URI(CONCAT("{local_uri}TI_", STRUUID())) AS ?timeInstant)
        }}
    }}
    {{?s ?p ?changeEvent}}UNION{{?changeEvent ?p ?o}}
}}
"""
print(query)

PREFIX agg: <http://jena.apache.org/ARQ/function/aggregate#>
PREFIX : <http://rdf.geohistoricaldata.org/address#>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX pr: <http://www.wikidata.org/prop/reference/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX data: <https://www.wikidata.org/wiki/Special:EntityData/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX pqn: <http://www.wikidata.org/prop/qualifier/value-normalized/>
PREFIX ns1: <http://rdf.geohistoricaldata.org/address#>
PREFIX spif: <http://spinrdf.org/spif#>
PREFIX path: <http://www.ontotext.com/path#>
PREFIX ref: <http://www.wikidata.org/reference/>
PREFIX pqv: <http://www.wikidata.org/prop/qualifier/value/>
PREFIX apf: <http://jena.apache.org/ARQ/property#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX rep: <http://www.openrdf.org/config/repository#>
PREFIX wgs: <http://www.